# Загрузка Pandas и очистка данных

In [1107]:
import pandas as pd
import ast
from bs4 import BeautifulSoup
import requests
import numpy as np
from multiprocessing import  Pool
from datetime import datetime, timedelta
from collections import namedtuple, Counter
from itertools import combinations
import random
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt

# this is just a file with a worker function for multiprocessing
# (otherwise multiprocessing doesn't work in Jupyter on Windows)
import worker  

%matplotlib inline

In [1108]:
RANDOM_SEED = 42

In [1109]:
# Helper functions are defined in this cell

'''
# These functions are in worker.py file now
def from_website(url):
    #print(f"https://www.tripadvisor.com{url}")
    r = requests.get(f"https://www.tripadvisor.com{url}", timeout=2)
    soup = BeautifulSoup(r.content)
    
    tmp = soup.find_all('span', {"class":"ratingDate"})
    rev_dates_lst = [i['title'] for i in tmp]
    #rev_dates_lst = [datetime.strptime(i['title'], '%B %d, %Y') for i in tmp]
    
    return rev_dates_lst


def apply_to_dataframe(df):
    df_result = df.copy()
    df_result['all_review_dates'] = df_result['URL_TA'].apply(from_website)
    return df_result
'''

def parallelize_dataframe(df, func, n_cores=30):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df


def create_freq_dict(d):
    '''
    Input dictionary must be sorted!
    
    This function takes dictionary with absolute frequencies as an argument
    and converts it into dictionary with relative frequencies percentages.
    
    For example, dictionary {'$$$$': 1423, '$': 6279, '$$ - $$$': 18412}
    will be turned into {'$$$$': 5.0, '$': 24.0, '$$ - $$$': 71.0}
    '''
    result = d
    tot = np.sum(list(d.values()))
    for k, v in d.items():
        d[k] = round(int(v) / tot * 100,0)
    
    return result


def fill_na_by_frequency(freq_dict, col):
    '''
    This function is used to replace NaN for categorical
    columns with values based on the frequency of non-missing values.

    For this function to work freq_dict (frequency dictionary) should be passed
    as an arguemnt. That's how frequency dictionary might look like (just an example)
    freq_dict: {'other': 7.0, 'father': 24.0, 'mother': 69.0}.
    Numbers correspond to the frequency of every possible distinct value.

    IMPORTANT: this function can't be applied using df.fillna() because the result
    must be different for every row it applies to, whereas fillna() replaces all NaN
    with a single value. Therefore apply() should be used on a column (series)
    '''

    rnd_check = round(random.random()*100)    
    
    cumulative = 0
    for k, v in freq_dict.items():
        # cumulative is used to correctly assess the probability
        # for example, if we take {'other': 7.0, 'father': 24.0, 'mother': 69.0}
        # then "other" will be picked is rnd_check if <= 7, "father" will be
        # picked if rnd_check <= 31 and "mother" will be picked in al the other cases
        cumulative += v
        if rnd_check <= cumulative:
            #print(f'Random number is: {rnd_check}. Value is: {k}')
            return k
    # the last option
    return k
    #print(f'Random number is: {rnd_check}. Value is: {k}')
    
    
def convert_price_range(s):
    '''
    Converts price ranges provided as one of these values
    $
    $$ - $$$            
    $$$$
    to low, average and high 
    '''
    if s == '$':
        return 'low'
    elif s == '$$ - $$$':
        return 'average'
    elif s == '$$$$':
        return 'high'
    else:
        return 'other'

def median_interval(l):
    '''
    Takes a list with dates as strings in a format like 'February 28, 2017'
    and returns a median interval between 2 consequent dates in a dataset
    '''   
    #tmp = ast.literal_eval(l)
    tmp = l
    res_lst = []
    if len(tmp) > 1:
        tmp = [datetime.strptime(i, '%B %d, %Y') for i in tmp]
        i = 0
        for d in tmp:
            res_lst.append((tmp[i] - d).days)
        return np.median(res_lst)
    return -1

def restaurant_age(l, snapshot_date):
    '''
    Takes a list with dates as strings in a format like 'February 28, 2017'
    and returns a day difference between snapshot_date and the first review date
    (let's  consider that an estimate of restaurant age on Tripadvisor website)
    '''   
    #tmp = ast.literal_eval(l)
    tmp = l
    res_lst = []
    if len(tmp) > 0:
        tmp = [datetime.strptime(i, '%B %d, %Y') for i in tmp]
        return (snapshot_date - np.min(tmp)).days
    return -1

def is_weekend(l):
    '''
    Takes a list with dates as strings in a format like 'February 28, 2017'
    and returns the most frequent weekday when the review was left
    '''   
    #tmp = ast.literal_eval(l)
    tmp = l
    res_lst = []
    if len(tmp) > 0:
        tmp = [datetime.strftime(datetime.strptime(i, '%B %d, %Y'),'%A') for i in tmp]
        day = Counter(tmp).most_common(1)[0][0]
        if day.lower() in ['saturday','sunday']:
            return 1
        else:
            return -1    
    return -1 

In [1110]:
df = pd.read_csv('main_task.xls')
df_small = df.iloc[:240,:] # I sometimes use it for testing
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [1111]:
df.head(10)

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963
5,id_1418,Oporto,NaN,1419.0,3.0,NaN,2.0,"[['There are better 3 star hotel bars', 'Amazi...",/Restaurant_Review-g189180-d12503536-Reviews-D...,d12503536
6,id_1720,Milan,"['Italian', 'Pizza']",1722.0,4.0,$,50.0,"[['Excellent simple local eatery.', 'Excellent...",/Restaurant_Review-g187849-d5808504-Reviews-Pi...,d5808504
7,id_825,Bratislava,['Italian'],826.0,3.0,NaN,9.0,"[['Wasting of money', 'excellent cuisine'], ['...",/Restaurant_Review-g274924-d3199765-Reviews-Ri...,d3199765
8,id_2690,Vienna,NaN,2692.0,4.0,NaN,NaN,"[[], []]",/Restaurant_Review-g190454-d12845029-Reviews-G...,d12845029
9,id_4209,Rome,"['Italian', 'Pizza', 'Fast Food']",4210.0,4.0,$,55.0,"[['Clean efficient staff', 'Nice little pizza ...",/Restaurant_Review-g187791-d8020681-Reviews-Qu...,d8020681


In [1112]:
'''
# Uncomment it only if you need to crawl Tripadvisor website once again
# Please note that it takes around 2-3 hours (depending on the machine)
# to perform the crawling
if __name__ ==  '__main__': 
    df_crawled = parallelize_dataframe(df, worker.apply_to_dataframe)
    
df_crawled.to_csv('with_additional_data_from_TA_ALL_2.csv')    
'''

# this is a file that consists of an original dataframe 
# extended with additional data from Tripadvisor website
# using requests and beautifulsoup
df = pd.read_csv('with_additional_data_from_TA_ALL_2.csv')
df.drop(labels=['Unnamed: 0'], axis=1, inplace=True)
df.head()

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,all_review_dates
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,"['February 14, 2020', 'December 20, 2019', 'No..."
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,[]
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,"['September 25, 2020', 'September 5, 2020', 'A..."
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,[]
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,"['September 10, 2020', 'September 28, 2019', '..."


In [1113]:
# Given the fact that it's currently year 2020 and a dataset for the task contains
# an older data it would be wise to avoid using records that appeared after a maximal
# date available in this dataset (the ones I obtained using web crawling).
# Let's find this date
df['tst'] = df['Reviews'].map(lambda x: x.split("], [")[1])
df['tst'] = df['tst'].str.replace('[','').str.replace(']','').str.replace("'",'').str.replace("'",'').str.split(',')
task_snapshot_date = pd.to_datetime(df['tst'].explode()).max()
df.drop(labels=['tst'], axis=1, inplace=True)
print(f"Snapshot date: {task_snapshot_date}")

Snapshot date: 2018-02-26 00:00:00


In [ ]:
# The data was saved to a file after web crawler (function from_website) was executed.
# It means that all_review_dates contains a string representation of a list.
# Let's convert it into the list to make other functions run faster
df['all_review_dates'] = df['all_review_dates'].map(ast.literal_eval)

# Let's remove all the review dates after task_snapshot_date
df['all_review_dates'] = df['all_review_dates'].map(lambda lst:\
    [i for i in lst if task_snapshot_date >= datetime.strptime(i, '%B %d, %Y')])

In [ ]:
# let's drop Restaurant_id, ID_TA and URL_TA because we don't need them anymore
df.drop(labels=['Restaurant_id', 'ID_TA','URL_TA'], axis=1, inplace=True)
df.head(5)

In [ ]:
# let's create 2 simple columns to mimic a basic sentiment analysis
# first column will be 1 if a restaurant contains some negative review
# (like bad, awful, horrible, dirty, disgusting) in 2 reviews provided
# in a dataset
bad_words = "terrible|very bad|awful|horrible|dirty|disgusting|bad experience|don't go|bad service|unfriendly|not good|"\
    "disappointed|rubbish|average food|bad customer service|poor"
df['contains_bad_review'] = df['Reviews'].str.contains(bad_words, regex=True, case=False).astype(int)

# same but with good words 
good_words = "good|nice|amazing|best|great|excellent"
df['contains_good_review'] = df['Reviews'].str.contains(good_words, regex=True, case=False).astype(int)

df.head()

In [ ]:
df[['contains_bad_review','contains_good_review']].sum()

In [ ]:
# there are some cases when no reviews are given in a list - let's mark them
df['no_reviews_in_list'] = (df['Reviews'] == "[[], []]").astype(int)

In [ ]:
# let's process dates available in 'Reviews' column
df['tst'] = df['Reviews'].map(lambda x: x.split("], ["))
df['dates'] = df['tst'].map(lambda x : x[1]).str.replace('[','').str.replace(']','').str.replace("'",'').str.split(',')
df['dates'] = df['dates'].map(lambda x: x if len(x) < 2 else [datetime.strptime(i.strip(), '%m/%d/%Y') for i in x])
# let's create a variable that stores number of days since TripAdvisor website creation (2004-02-01) up to the most recent comment
# if there's no  date available - mark this column as -1
df['days_since_ta_creation'] = df['dates'].map(lambda x : -1 if len(x) < 2 else (max(x) - datetime.strptime('2004-02-01', '%Y-%m-%d')).days)
# let's create a variable that stores days between comments
df['days_between_comments'] = df['dates'].map(lambda x : -1 if len(x) < 2 else (x[0] - x[1]).days)
df.drop(labels=['tst','dates'], axis=1, inplace=True)

In [ ]:
# determining unique cuisine styles and their value counts
# ast.literal_eval is required because lists of styles are
# represented as strings in the dataframe
# IMPORTANT: nan values are replaced by the fake list empty list "[]" for this
# particular part of the task, but that does not happen in place
df['cuisine_styles_num'] = df['Cuisine Style'].fillna("[]").apply(ast.literal_eval).apply(len)

In [ ]:
# let's check the number of unique cuisine styles (empty records will be marked as
# a "no_style_provided" in place)
df['Cuisine Style'].fillna("['no_style_provided']", inplace=True)

unique_styles = df['Cuisine Style'].apply(ast.literal_eval).explode()\
    .value_counts(normalize=True)

sum(unique_styles.head(25)), unique_styles.head(25)

In [ ]:
# Let's take top 25 styles and create binary variables out of them
# (for example, European can be either 0 or 1).
for c in list(unique_styles.head(25).index):
    df[c] = df['Cuisine Style'].str.contains(c, regex=False).astype(int)
df.head(25)

In [ ]:
# let's consider NaN in number of reviews as 0
df['Number of Reviews'].fillna(0, inplace=True)
df.info()

In [ ]:
df['City'].nunique()

In [ ]:
# Let's use worldcities dataset from https://www.kaggle.com/viswanathanc/world-cities-datasets
# I want to create 2 dimension from an original 'City' column: capital and  population
df_cities = pd.read_csv('worldcities.csv')

# leaving only required columns from worldcities data set
df_cities = df_cities[['city_ascii','capital','population']]

# the assumption here is that most of the capitals will be listed as primary
# and we don't really care about other classification because the variable will be
# binary in the end (capital = 0 or 1)
df_cities['capital'].fillna('other', inplace=True) 

# the assumption here is that most important cities will have the population
# mentioned in the data set
df_cities['population'].fillna(0, inplace=True)

# leaving only cities with max population - this will eliminate duplicates
# in case multiple cities with the same name exist
df_cities.sort_values(by=['city_ascii','population'], ascending=[True,False], inplace=True)
df_cities.drop_duplicates(subset='city_ascii', keep='first', inplace=True)

# making sure we have no more duplicates
df_cities['city_ascii'].value_counts()

In [ ]:
# modifying 'capital' column to make it binary (if capital then 1 else 0)
df_cities['capital'] = df_cities['capital'].map(lambda x : 1 if x == 'primary' else 0)
df_cities['capital'].value_counts()

In [ ]:
# merging df with df_cities based on city name
df = df.merge(df_cities, how='left', left_on='City', right_on='city_ascii')
df.info()

In [ ]:
# let's see what cities are missing
df[df['capital'].isna()]['City'].value_counts()

In [ ]:
# it seems that we're missing the data about only one city - Oporto (aka Porto in Portugal)
# let's locate it and populate missing values manually
df['capital'].fillna(0, inplace=True)
df['population'].fillna(1337000, inplace=True)

In [ ]:
# let's encode cities
df['City_copy'] = df['City']
df = pd.get_dummies(df, columns=[ 'City_copy',], dummy_na=True)

In [ ]:
# let's drop some columns that are not required anymore
# to make the dataframe more compact and readable
df.drop(['city_ascii', 'Cuisine Style', 'Reviews'], axis=1, inplace=True)
df.info()

In [ ]:
# let's take a look at price ranges
df['Price Range'].value_counts()

In [ ]:
# it seems there are many NaN values among price ranges
# let's replace NaN with values based on a frequency of the existing values
# custom function will be used for that (see definition of helper functions above)

# please note that ascending=True is important here because the dictionary must be sorted
freq_dict = create_freq_dict(dict(df['Price Range'].value_counts(ascending=True)))
df['Price Range'] = df['Price Range'].apply(
    lambda x: fill_na_by_frequency(freq_dict, "Price Range") if pd.isna(x) else x)
# also let's make Price Range names more meaningful (before we one hot encode them)
df['Price Range'] = df['Price Range'].apply(convert_price_range)

In [ ]:
# let's take a look at price ranges once again - it seems that there are
# no more missing values
df['Price Range'].value_counts()

In [ ]:
# and there are no more missing values in the dataframe as such
df.info()

In [ ]:
# processing Price Range with one hot encoder
enc = OneHotEncoder(handle_unknown='ignore')
y = OneHotEncoder().fit_transform(df['Price Range'].to_numpy().reshape(-1, 1)).toarray()
# list(df['Price Range'].unique()) is safe because the values are provided 
# in order of appearance
df_price_rng = pd.DataFrame(y, columns=list(df['Price Range'].unique()))
df = pd.concat([df,df_price_rng], axis=1)
df.drop(['Price Range'], inplace=True, axis=1)

In [ ]:
# let's calculate restaurant_age for every restaraunt and drop 'all_review_dates' column
df['restaurant_age'] = df['all_review_dates'].map(lambda x: restaurant_age(x, snapshot_date=task_snapshot_date))
# let's calculate median interval between consequetive review dates
df['median_interval'] = df['all_review_dates'].map(median_interval)
# let's determine if most reviews were left on weekend or a weekday
df['is_weekend'] = df['all_review_dates'].map(is_weekend)
df.drop(labels=['all_review_dates'], axis=1, inplace=True)
df.info()
print(f"Most popular restaurant age on a tripadvisor: {df['restaurant_age'].value_counts().index[0]}")

In [ ]:
# visual assessment of numeric variables
for x in (df['City'].value_counts())[0:10].index:
    df['Ranking'][df['City'] == x].hist(bins=100)
plt.show()

In [ ]:
# let's create a variable that shows if restaraunt is above or below the average Ranking in the city
df['avg_rank_by_city'] = df.groupby('City')['Ranking'].transform(np.mean)
df['better_than_average'] = (df['Ranking'] < df['avg_rank_by_city']).astype(int)

# let's normalize Ranking using count of restaurants in the city
df['cnt_rest_in_the_city'] = df.groupby('City')['City'].transform('count')
df['ranking_norm'] = df['Ranking']/df['cnt_rest_in_the_city']

# let's normalize Ranking using max rank by city
df['max_rank_by_city'] = df.groupby('City')['Ranking'].transform(np.max)
df['ranking_norm_max'] = df['Ranking']/df['max_rank_by_city']

# let's normalize Ranking using count of reviews by city
df['cnt_rev_by_city'] = df.groupby('City')['Number of Reviews'].transform('sum')
df['ranking_norm_reviews'] = df['Ranking']/df['cnt_rev_by_city']

In [ ]:
for x in (df['City'].value_counts())[0:10].index:
    df['ranking_norm'][df['City'] == x].hist(bins=100)
plt.show()

In [ ]:
for x in (df['City'].value_counts())[0:10].index:
    df['ranking_norm_max'][df['City'] == x].hist(bins=100)
plt.show()

In [ ]:
for x in (df['City'].value_counts())[0:10].index:
    df['ranking_norm_reviews'][df['City'] == x].hist(bins=100)
plt.show()

In [ ]:
# let's create a variable that shows number of reviews per 1000 citizens
df['reviews_per_capita'] = df['Number of Reviews']*1000 / df['population']

In [ ]:
df.drop(['City', 'avg_rank_by_city', 'cnt_rest_in_the_city', 'cnt_rev_by_city', 'max_rank_by_city'], axis=1, inplace=True)

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [ ]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
#X = df.drop(['Restaurant_id', 'Rating'], axis = 1)
X = df.drop(['Rating'], axis = 1)
y = df['Rating']

In [ ]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [ ]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RANDOM_SEED)

# Создаём, обучаем и тестируем модель

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100, random_state=RANDOM_SEED)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))